In [1]:
import sys
import os
import pandas as pd
homey = os.getcwd()
redouxPath = os.path.join(homey, 'ForecastRedoux')
rawDataPath = os.path.join(redouxPath, 'RawData')
AdditionalInfoPath = os.path.join(homey, 'AdditionalInfo')
sys.path.insert(0, redouxPath)
import ForecastMain as fm
import ForecastTimelineBackend as ftlb
import ForecastSettings as fs
import xlsxwriter
import datetime as dt

In [2]:
missingboms = fs.No_BOMs()  # Creates an instance of the class in Forecast Settings No_BOMs
manyboms = fs.Many_BOMs()  # Creates an instance of the class in Forecast Settings Many_BOMs
datalist = fm.data_prep()

In [3]:
datalist[0] = fm.stitch_builds_to_orders(datalist[0].copy(), datalist[2].copy(), missingboms, manyboms)
print('Imaginary builds added ...')

Imaginary builds added ...


In [4]:
normal_orders = [datalist[0], datalist[1]]  # newordersdf, invdf
invdf = datalist[1]
startinginvdf = datalist[1].copy() # invdf is going to change throughout, this is a reference for adding and inventory counter to the timeline later
bomsdf = datalist[2]
mypartsdf = fm.gather_parts()

In [5]:
mytierlist = ftlb.create_bom_tiers_v2(bomsdf, mypartsdf)

section 1
section 2
tier 2 created
tier 3 created
tier 4 created
tier 5 created
tier 6 created
tier 7 created
tier 8 created
tier 9 created
section 3


In [6]:
tier = 1
while len(mytierlist) > 0:
    #print('Running tier', tier)
    tierlist = mytierlist[tier]
    normal_orders = fm.complete_orders_loop_redux(tierlist, normal_orders[0], invdf, bomsdf, missingboms, manyboms)
    normal_orders[0] = normal_orders[0].append(normal_orders[2])
    del mytierlist[tier]
    tier += 1
print('*Timeline Has Been Created*')

*Timeline Has Been Created*


In [7]:
timingtest = ftlb.find_timing_issues(normal_orders[0], normal_orders[1])

In [9]:
timeline = normal_orders[0].copy()

In [15]:
phantomtimelineP = timeline.ix[timeline['ITEM'] == 'Phantom']
phantomtimelineI = timeline.ix[timeline['ITEM'] == 'Imaginary']

In [17]:
phantomtimeline = phantomtimelineP.copy().append(phantomtimelineI.copy())

In [19]:
index = 4


In [27]:
phantomtimeline.ix[4]

,DATESCHEDULED,ITEM,Make/Buy,ORDER,ORDERTYPE,PARENT,PART,QTYREMAINING
4,2017-02-22 00:00:00,Phantom,Buy,2647,Purchase,48809/42018,010-531-10,1.0
4,2017-02-28 00:00:00,Phantom,Buy,2380,Raw Good,432,030-1224-10 r02,-4.0


In [29]:
normal_orders[0][normal_orders[0]['PART'] == '030-1229-10 r04']

,DATESCHEDULED,ITEM,Make/Buy,ORDER,ORDERTYPE,PARENT,PART,QTYREMAINING
27,2016-08-12 13:16:00,23209,Buy,23209:019,Raw Good,23209:019,030-1229-10 r04,-5.0
28,2016-10-26 00:00:00,Phantom,Buy,154,Raw Good,45623,030-1229-10 r04,-46.0
29,2016-11-18 11:39:00,23883,Buy,23883:012,Raw Good,23883:012,030-1229-10 r04,-1.0
30,2017-01-30 00:00:00,Phantom,Buy,336,Raw Good,48168,030-1229-10 r04,-32.0
31,2017-02-08 08:40:00,24196,Buy,24196:006,Raw Good,24196:006,030-1229-10 r04,-2.0
32,2017-02-28 00:00:00,Phantom,Buy,403,Raw Good,47433,030-1229-10 r04,-8.0
33,2017-03-01 00:00:00,Phantom,Buy,432,Raw Good,47752,030-1229-10 r04,-4.0
34,2017-03-15 09:54:00,24363,Buy,24363:001,Raw Good,24363:001,030-1229-10 r04,-24.0
35,2017-03-28 10:04:00,24154,Buy,24154:035,Raw Good,24154:035,030-1229-10 r04,-17.0
36,2017-05-14 00:00:00,Imaginary,Buy,-2188,Purchase,-16,030-1229-10 r04,100.0


In [30]:
normal_orders[0].ix[35]

,DATESCHEDULED,ITEM,Make/Buy,ORDER,ORDERTYPE,PARENT,PART,QTYREMAINING
35,2017-02-20 00:00:00,16,Buy,48745,Sale,48745,020-123-10,-1.0
35,2017-03-28 10:04:00,24154,Buy,24154:035,Raw Good,24154:035,030-1229-10 r04,-17.0


In [34]:
timeline = timeline.copy().sort_values(by=['PART', 'DATESCHEDULED', 'ORDER'], ascending=[True, True, True])

In [38]:
timeline.reset_index(drop=True, inplace=True)